In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image
from IPython.core.display import display
from sklearn.tree.export import export_graphviz

import matplotlib.pyplot

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles

In [3]:
import random

# データセットの入力

In [4]:
#irisデータ 
#ここでデータ準備する
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data)
y=pd.DataFrame(cancer.target , columns = [30])
Dataset = pd.concat([X,y],axis=1)
Dataset_index = pd.DataFrame(Dataset.index)

In [5]:
Loop = 31 #繰り返し回数

# ブーストラップ標本、out-of-bag標本の作製

In [6]:
class Boostrap:
    """Bagging data class """
    def __init__(self,Dataset):
        #コンストラクタ
        self.Dataset = Dataset;
        self.Loop = Loop;
        self.D_bs = D_bs;
        self.Test_bs = Test_bs;
        
    def boostrap_data(self):
            #ブーストラップ標本の作製
            D_bs_index =np.random.choice(Dataset.index , len(Dataset.index) , replace =True)
            #ランダムにindexを取得　569個のデータを重複ありで抽出する
            d_bs = np.zeros((len(Dataset.index),len(Dataset.columns)+1))
            D_bs = pd.DataFrame(d_bs)
               
            #D_bsに格納　
            for i in range(len(Dataset.index)):
                for j in range(len(Dataset.columns)):
                    D_bs[0][i] = D_bs_index[i]
                    D_bs[j+1][i] = Dataset.iloc[D_bs_index[i]:D_bs_index[i]+1,0:len(Dataset.columns)][j]

            #out-of-bag事例の作成
            A = Dataset.index
            A_list = set(A)
            D_bs_index_list = set(D_bs_index)
            #set型にして比較しやすいようにした

            B= A_list - D_bs_index_list #比較　重複を消す
            B = list(B) #list型に直す

            test_bs = np.zeros((len(B),len(Dataset.columns)+1))
            Test_bs = pd.DataFrame(test_bs)

            #Test_bsに格納　Test_bs作成
            for i in range(len(B)):
                for j in range(len(Dataset.columns)):
                    Test_bs[0][i] = B[i]
                    Test_bs[j+1][i] = Dataset.iloc[B[i]:B[i]+1,0:len(Dataset.columns)][j]
                #index 0 id 1=30 特徴量　31ターゲット（ラベル）
            return [D_bs ,Test_bs]

train ,test  = Boostrap.boostrap_data(Dataset)

# 学習

In [7]:
from sklearn import tree

In [8]:
class Bagging:
    """Bagging result"""
    def __init__(self,train,test):
        self.train = train;
        self.test = test;
    #コンストラクタ
        
    def bagging_fit(self):
        
        #訓練データの特徴量とtargetに分類
        train_feature = train.iloc[:,1:31] #特徴量
        train_target = train.iloc[:,31]    #ラベル
        train_index = train.iloc[:,0:1]    #データid
        
         #決定木の作成,out-of-bag予測　（3.4）式
        clf = tree.DecisionTreeClassifier(max_depth=10)
        clf = clf.fit(train_feature, train_target)
        
        return clf
    
    def bagging_predict(self):
        #テストデータ(out-of-bag標本)の特徴量とtargetに分類
        test_feature = test.iloc[:,1:31]  #特徴量
        test_target = test.iloc[:,31]     #ラベル
        test_index = test.iloc[:,0:1]     #データid
        #フィッティングして予測する out-of-bag予測　（3.4）式
        
        clf = Bagging.bagging_fit(train) #フィッティング
        predicted = clf.predict(test_feature) #予測
        err = 1-(sum(predicted == test_target) / len(test_target))  #それぞれのブーストラップ標本の誤差率
        
        #out-of-bag事例で予測し、out-of-bag事例のデータid毎の予測ラべル
        # index 0 out-of-bag事例のデータid , 1 out-of-bag事例のデータid毎のラベル
        H_obb_predicted = pd.concat([test_index,pd.DataFrame(predicted)],axis =1) 
        H_obb_predicted.columns =[0,1]
        
        print(err)
        return [err,H_obb_predicted]

error,predicted  =  Bagging.bagging_predict(test)

# 実際は複数回loopするので…

In [9]:
error = np.empty(Loop)    #それぞれのブーストラップ標本の誤差率
#投票（多数決）によるクラス分類を格納 
#0 id 1=(Loop-1):繰り返した際のそれぞれのブーストラップ標本でのout-of-bag予測のラベルを格納している Loop 多数決でのクラス
Voting = np.full((len(Dataset.index),Loop+2),10) 
Voting = pd.DataFrame(Voting)
for i in range(len(Dataset.index)):
    Voting[0][i] = Dataset.index[i]

In [10]:
#Loop 繰り返してブーストラップ標本、out-of-bag標本の作製
#それぞれで誤差率評価　かつ、クラス分類
for loop in range(Loop):
    train ,test = Boostrap.boostrap_data(Dataset)
    err, H_obb_predicted = Bagging.bagging_predict(test)
    error[loop] = err
    
    #Votingに格納していく
    for i in range(len(Dataset.index)):
        for j in range(len(H_obb_predicted)):
            if(Voting[0][i] == H_obb_predicted[0][j]):
                Voting[loop+1][i] =  H_obb_predicted[1][j]

0.0555555555556
0.0755555555556
0.0588235294118
0.0663507109005
0.101851851852
0.0651162790698
0.066985645933
0.105769230769
0.075
0.0612244897959
0.117073170732
0.0935960591133
0.0803571428571
0.0497512437811
0.0869565217391
0.0681818181818
0.0809523809524
0.0888888888889
0.0943396226415
0.0462962962963
0.0645161290323
0.0809523809524
0.0490196078431
0.0921658986175
0.0769230769231
0.0622009569378
0.0628019323671
0.0691244239631
0.0487804878049
0.0990566037736
0.0849056603774


In [12]:
#compare Votingのidないver　比較用に作成
compare = Voting.iloc[0:len(Dataset.index),1:]
#count それぞれのidで比較するため用
count = np.empty(len(Dataset.index))
for i in range(len(Dataset.index)):
    for j in range(Loop):
        if(compare[j+1][i] ==0):
            count[i] = count[i]+1 #それぞれのidでの0の数をカウントしている

#多数決
for i in range(len(Dataset.index)):
    if(count[i] >= float(Loop) / 2):
        Voting[Loop+1][i] = 0
    else:
        Voting[Loop+1][i] = 1
        
result =  pd.concat([Dataset[30] ,Voting[Loop+1]] , axis =1)

#全体の誤差
Count =0
for i in range(len(Dataset.index)):
    if(result[30][i] == result[Loop+1][i]):
        Count = Count +1 #Countは正答率

H_obb_error = ((len(Dataset.index)-Count)/(len(Dataset.index)))
print(H_obb_error)

0.34973637961335674


In [19]:
result

,30,32
0,0,1
1,0,1
2,0,1
3,0,1
4,0,0
5,0,1
6,0,1
7,0,1
8,0,1
9,0,1
